In [1]:
from IPython.core.display import display, HTML
display(HTML("""<style>#notebook-container{width: 100%}.code_cell {flex-direction: row !important;}.code_cell .output_wrapper {width: 40%}.code_cell .input {width: 60%}</style>"""))

/tmp/ipykernel_18118/4130883065.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

# Create Model

In [3]:
class Neural(nn.Module):
    def __init__(self):
        super(Neural, self).__init__()
        self.seq = nn.Sequential(
            nn.Linear(3, 100),
            nn.Tanh(),
            nn.Linear(100, 3)
            )
    
    def forward(self,x):
        x = self.seq(x)
        return x

In [4]:
model = Neural()
print(model)

Neural(
  (seq): Sequential(
    (0): Linear(in_features=3, out_features=100, bias=True)
    (1): Tanh()
    (2): Linear(in_features=100, out_features=3, bias=True)
  )
)


# Test Input and Output with Fake data

In [5]:
input_fake = torch.randn(10,3)
print(input_fake)
print(input_fake.size())

tensor([[ 0.5546, -0.3670, -1.1554],
        [ 0.1488,  1.6004, -2.2068],
        [ 1.3327,  0.3522,  1.4457],
        [ 0.2227, -0.5922, -1.4854],
        [ 0.3382,  0.2373,  0.3203],
        [-0.5176, -1.9909,  0.5522],
        [ 1.2672,  0.0404, -1.5949],
        [ 0.8923, -0.4759,  2.0397],
        [-1.3051, -1.1264, -0.7466],
        [ 1.0795,  0.5210,  0.1983]])
torch.Size([10, 3])


In [6]:
output_fake = model(input_fake)
print(output_fake)
print(output_fake.size())

tensor([[-0.5588, -0.1495,  0.0231],
        [ 0.1199,  0.0157,  0.0039],
        [-0.2212, -0.5168,  0.7484],
        [-0.5163, -0.0602, -0.1423],
        [-0.2164, -0.3114,  0.3549],
        [-0.4352, -0.0682, -0.2357],
        [-0.6594, -0.1948,  0.1347],
        [-0.2498, -0.4162,  0.7020],
        [-0.1219,  0.0868, -0.4692],
        [-0.3632, -0.4102,  0.5335]], grad_fn=<AddmmBackward0>)
torch.Size([10, 3])


# Generating Data from Forward Kinematics
$ x_E = l_1 cos(q_1) + l_2 cos(q_1 + q_2) + l_3 cos(q_1 + q_2 + q_3) $

$ y_E = l_1 sin(q_1) + l_2 sin(q_1 + q_2) + l_3 sin(q_1 + q_2 + q_3) $

$ \theta_E = q_1 + q_2 + q_3 $

$ l_1 = l_2 = l_3 = 2, q_1 \in [0,\pi], q_2 \in [-\pi,0], q_3 \in [-\pi/2,\pi/2]$

In [7]:
l1 = l2 = l3 = 2

q1 = np.linspace(0, np.pi,1000)
q2 = np.linspace(-np.pi, 0,1000)
q3 = np.linspace(-np.pi/2, np.pi/2, 1000)
print("q1"),print(np.shape(q1))
print("q2"),print(np.shape(q2))
print("q3"),print(np.shape(q3))

q1
(1000,)
q2
(1000,)
q3
(1000,)


(None, None)

In [8]:
x = l1*np.cos(q1) + l2*np.cos(q1+q2) + l3*np.cos(q1+q2+q3)
y = l1*np.sin(q1) + l2*np.sin(q1+q2) + l3*np.sin(q1+q2+q3)
t = q1+q2+q3

In [9]:
print(np.shape(x))
print(np.shape(y))
print(np.shape(t))

(1000,)
(1000,)
(1000,)


In [10]:
x = np.expand_dims(x, axis=1)
y = np.expand_dims(y, axis=1)
t = np.expand_dims(t, axis=1)

In [11]:
print(np.shape(x))
print(np.shape(y))
print(np.shape(t))

(1000, 1)
(1000, 1)
(1000, 1)


In [12]:
x = x.transpose()
y = y.transpose()
t = t.transpose()

print(np.shape(x))
print(np.shape(y))
print(np.shape(t))

(1, 1000)
(1, 1000)
(1, 1000)


In [13]:
g = np.append(x,y,axis=0)
h = np.append(g,t,axis=0)

In [14]:
print(h)

[[-3.67394040e-16 -1.88384765e-02 -3.76159390e-02 ... -4.03753682e+00
  -4.01881870e+00 -4.00000000e+00]
 [ 2.00000000e+00  1.99362159e+00  1.98706562e+00 ... -1.96190789e+00
  -1.98104267e+00 -2.00000000e+00]
 [-4.71238898e+00 -4.70295477e+00 -4.69352056e+00 ...  4.69352056e+00
   4.70295477e+00  4.71238898e+00]]


In [15]:
print(np.shape(h))
h = h.transpose()
print(np.shape(h))

(3, 1000)
(1000, 3)


In [16]:
input_real = torch.tensor(h)

In [17]:
print(input_real.size())

torch.Size([1000, 3])


In [18]:
output_real = model(input_real.float())

In [19]:
print(output_real.size())

torch.Size([1000, 3])


# Create Ground Trust

In [20]:
q1 = np.expand_dims(q1, axis=1)
q2 = np.expand_dims(q2, axis=1)
q3 = np.expand_dims(q3, axis=1)

In [21]:
print("q1"),print(np.shape(q1))
print("q2"),print(np.shape(q2))
print("q3"),print(np.shape(q3))

q1
(1000, 1)
q2
(1000, 1)
q3
(1000, 1)


(None, None)

In [22]:
q1 = q1.transpose()
q2 = q2.transpose()
q3 = q3.transpose()

print(np.shape(q1))
print(np.shape(q2))
print(np.shape(q3))

(1, 1000)
(1, 1000)
(1, 1000)


In [23]:
p = np.append(q1,q2,axis=0)
o = np.append(p,q3,axis=0)

[[ 0.          0.00314474  0.00628947 ...  3.13530318  3.13844792
   3.14159265]
 [-3.14159265 -3.13844792 -3.13530318 ... -0.00628947 -0.00314474
   0.        ]
 [-1.57079633 -1.56765159 -1.56450685 ...  1.56450685  1.56765159
   1.57079633]]


In [24]:
print(np.shape(o))
o = o.transpose()
print(np.shape(o))

(3, 1000)
(1000, 3)


In [25]:
ground_trust = torch.tensor(o)

In [33]:
from torch.utils.data import Dataset, DataLoader
train_loader = DataLoader(input_real, batch_size=10, shuffle=True)
print(len(train_loader))

100


In [31]:
#init loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params = model.parameters(),lr = 0.0003)

In [ ]:
#train
for epc in range(10):
    model.train()
    num_cor = 0
    num_samp = 0
    loss_list = []
    for data,target in dataloader:
        data = data.to("cuda")
        target = target.to("cuda")

        #forward
        output = model(data)
        loss = criterion(output,target)
        loss_list.append(loss)
        
        _,pred = output.max(1)

        num_cor += (pred == target).sum()
        num_samp += pred.size(0)
        

        #backward
        optimizer.zero_grad()
        loss.backward()

        #grad decent or adam
        optimizer.step()
    acc = num_cor/num_samp
    print("EPC : "+str(epc)+"   Acc:  "+str(acc))